<a href="https://colab.research.google.com/github/adalbertii/Modele-regresyjne/blob/main/pipeline-boston-house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Model regresyjny**

Oparty na sieciach neuronowych

1.   Ocena modeli sieci neuronowych z punktu widzenia ich wewnętrznej topologii
2.   Badany jest równiez wpływ warstwy standaryzującej
3.   Prezentowanana jest koncepcja budowy PIPELINE

In [4]:
! pip install Keras

In [5]:
! pip install scikeras

In [6]:

from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [20]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
datframe = read_csv(url)
#datframe = read_csv('boston.csv')

In [ ]:
datframe.head()


In [ ]:
datframe.info()

In [23]:
# konwersja dataframe do typu tablicowego

dataset = datframe.values

In [ ]:
dataset.shape

In [25]:
# podział zbioru danych na zbirr ze zmiennymi objaśniajacymi [X] i zmienną objasnianą [Y]
X = dataset[:,:-1]
Y = dataset[:,-1]

In [ ]:
Y.shape

In [ ]:
X.shape

In [28]:
# definicja procedury przeznaczonej do budowy i kompilacji modelu sieci neuronowej
def baseline_model():

 model = Sequential()
 # parametr input_sape jest taki sam jak kształt tablicy X
 model.add(Dense(13, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))

 # kompilacja modelu
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


**Ewaluacja modelu**
oparatA na walidacji krzyżowej , z podziałem zbioru na 10 podzbiorów

In [30]:
estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)

In [31]:
results_baseline_model = cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_squared_error')


In [32]:
print("Model podstawowy: %.2f (%.2f) MSE" % (results_baseline_model.mean(), results_baseline_model.std()))

Model podstawowy: -33.84 (24.99) MSE


**Budowa modelu z warstwą standaryzującą dane wejściowe do modelu **

In [33]:
# DEFINICJA PROCEDURY BUDOWY MODELU
def baseline_model():

 model = Sequential()
 model.add(Dense(13, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))
 # kompilacja  modelu
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [35]:
# budowa pipeline ze standaryzacją danych wejściowych i budową, uczeniem i ewaluacją modelu

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=baseline_model, epochs=50, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)


In [ ]:
results_baseline_model_with_st = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Model podstawowy z warstwą standaryzującą: %.2f (%.2f) MSE" % (results_baseline_model_with_st.mean(), results_baseline_model_with_st.std()))

**Dostrajanie topologii sieci neuronowej**

Model sieci neuronowej można zoptymalizować pod wieloma względami.

Być może punktem największego wpływu jest struktura samej sieci, w tym liczba warstw i liczba neuronów w każdej warstwie.

W tej sekcji zostaną ocenione dwie dodatkowe topologie sieci w celu dalszej poprawy wydajności modelu.

Przyjrzymy się zarówno głębszej, jak i szerszej topologii sieci.



**Ocena głębszej topologii sieci**

Jednym ze sposobów poprawy wydajności sieci neuronowej jest dodanie większej liczby warstw. Może to pozwolić modelowi na wyodrębnienie i ponowne połączenie cech wyższego rzędu osadzonych w danych.

W tej sekcji oceniony zostanie efekt dodania do modelu jeszcze jednej warstwy ukrytej. Jest to tak proste, jak zdefiniowanie nowej funkcji do tworzenia tego głębszego modelu, skopiowanego z powyższego modelu bazowego. Następnie można wstawić nową linię po pierwszej warstwie ukrytej - w tym przypadku z około połową liczby neuronów

In [37]:
# procedura budowy głębszej sieci neuronowej
def larger_model():


 model = Sequential()
 model.add(Dense(13, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(6, kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))

 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


In [38]:
# evaluacja  głębszego modelu z warstwą standaryzującą
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)


In [39]:
results_larger_model_with_std = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Głębszy model z warstwą standaryzującą : %.2f (%.2f) MSE" % (results_larger_model_with_std.mean(), results_larger_model_with_std.std()))

Głębszy model z warstwą standaryzującą : -24.73 (29.51) MSE


**Ocena szerszej topologii sieci**

Innym podejściem do zwiększenia możliwości reprezentacyjnych modelu jest stworzenie szerszej sieci.

W tej sekcji ocenimy efekt zachowania płytkiej architektury sieci i niemal podwojenia liczby neuronów w jednej warstwie ukrytej.

 W tym przypadku liczba neuronów w warstwie ukrytej zostanie zwiększona z 13 do 20 w porównaniu do modelu bazowego.

In [ ]:

def wider_model():

 model = Sequential()
 model.add(Dense(20, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))

 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


In [ ]:
# evaluacja szerszego modelu
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)


In [ ]:
results = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Szerszy model z warstwą standaryzującą: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -21.54 (23.94) MSE


In [ ]:
print("Model podstawowy                         : %.2f (%.2f) MSE" % (results_baseline_model.mean(), results_baseline_model.std()))
print("Model podstawowy z warstwą standaryzującą: %.2f (%.2f) MSE" % (results_baseline_model_with_st.mean(), results_baseline_model_with_st.std()))
print("Głębszy model z warstwą standaryzującą   : %.2f (%.2f) MSE" % (results_larger_model_with_std.mean(), results_larger_model_with_std.std()))
print("Szerszy model z warstwą standaryzującą   : %.2f (%.2f) MSE" % (results.mean(), results.std()))